## Model Training and Evaluation

In [ ]:
import pandas as pd
import numpy as np
import mlflow
import tempfile
import IPython.display
import pathlib
import pickle

import matplotlib.pyplot as plt

import sklearn
import sklearn.model_selection
import sklearn.impute
import sklearn.ensemble
import sklearn.pipeline
import sklearn.tree
import sklearn.metrics
import sklearn.neural_network
import sklearn.preprocessing
import sklearn.compose

import ml_colon
import ml_colon.data_preparation

In [ ]:
trained_models = []

### Retrieving Data

We have implemented the data cleaning the `ml_colon.data_preparation` module and with that retrieve the "cleaned" DataFrame. By "cleaned" we mean that we have filtered out all rows that we want to exclude from training. No further rows will be excluded from here onwards. 

In [ ]:
df = ml_colon.data_preparation.get_clean_df_from_csv()

print(f"Loaded data set with {len(df)} rows.")

### Splitting Train / Test set

Next we split the data set into the train / test set.

In [ ]:
test_size = 0.2 # 20% of rows
features = [c for c in df.columns if c != ml_colon.TARGET_VARIABLE]

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    df[features],
    df[ml_colon.TARGET_VARIABLE],
    test_size=test_size,
    random_state=ml_colon.SEED,
)

print(f"Train set: {len(y_train)} rows | Test set: {len(y_test)} rows\n")
print(f"Train set grouped by relevant: \n{y_train.value_counts()}\n")
print(f"Test set grouped by relevant: \n{y_test.value_counts()}")

#### Upsampling

Where are dealing with an imbalanced data set regarding the target variable. Any estimator can reach 82% accuracy by simply always predicting 1.
Reminder that:
\begin{equation}
accuracy = \dfrac{TP + TN}{P + N} 
\end{equation}

Therefore we need to upsample the rows in our dataframe with relevant = 0

In [ ]:
def upsample(df: pd.DataFrame, n: int) -> pd.DataFrame:
    """Upsample the DataFrame with respect to the target variable "relevant".

    Parameters
    ----------
    df : pd.DataFrame
        [description]
    n : int
        number of rows in resulting DataFrame

    Returns
    -------
    pd.DataFrame
        the upsampled DataFrame
    """
    weight_relevant_1 = 0.5 / len(df[df.relevant == 1] )
    weight_relevant_0 = 0.5 / len(df[df.relevant == 0] )

    relevant_weighted = df.relevant.replace({
        0: weight_relevant_0,
        1: weight_relevant_1
    })
    return df.sample(
        n=n,
        replace=True,
        weights=relevant_weighted
    )

In [ ]:
train_df = X_train
train_df[ml_colon.TARGET_VARIABLE] = y_train

train_df = upsample(train_df, n=len(X_train))

X_train = train_df[features]
y_train = train_df[ml_colon.TARGET_VARIABLE]

print(f"Train set: {len(y_train)} rows")
print(f"Train set grouped by relevant: \n{y_train.value_counts()}\n")

### Classifiers and Parameters Definition
We will try different models to get the best outcome with different hyperparameters based on the accuracy of the model. The different classifiers used are: 

- K-Nearest Neighbor
- Random Forest 
- Multi-Layer Perceptron

With this selection, there is a variety of complexity of models used. There is one simple model, namely the K-Nearest Neighbor algorithm. The Random Forest algorithm is an ensemble method and a more powerful method than K-Nearest Neighbors. Nowadays, there is a big hype around neural networks and their power of finding interesting patterns in data, therefore the Multi-Layer Perceptron is included. 

Each of these algorithms will be tested with different hyperparameters using grid search. The different values of the parameters are described below.

In [ ]:
score_metric = "accuracy"
assert score_metric in ml_colon.SCORE_METRICS

In [ ]:
implemented_classifiers = {
    "k_neighbor": {
        "classifier": sklearn.neighbors.KNeighborsClassifier(),
        "param_grid": [{"classifier__n_neighbors": [5, 11, 15]}],
    },
    "random_forest": {
        "classifier": sklearn.ensemble.RandomForestClassifier(max_features=2),
        "param_grid": [{"classifier__max_depth": [4, 8, 10, 12, 14, 16, 18, 20], "classifier__n_estimators": [10, 15, 20, 25, 30, 35, 40, 45, 50, 55]}],
    },
    "multilayer_perceptron": {
        "classifier": sklearn.neural_network.MLPClassifier(),
        "param_grid": [
            {"classifier__alpha": [0.001, 0.01, 0.1, 0.5], "classifier__activation": ["identity", "relu"]}
        ],
    },
}

classifier = "k_neighbor"
assert classifier in implemented_classifiers

#### Model Pipeline

Before a machine learning model can be trained some data transformations need to be done. For that we use a `sklearn.pipeline.Pipeline` to chain the data transformations such as imputing missing values or scaling.

In [ ]:
index_continous_columns = X_train.columns.get_indexer(
    X_train.select_dtypes(include=np.float).columns
)

In [ ]:

pipeline = sklearn.pipeline.Pipeline(
    steps=[
        (
            "imputer", sklearn.impute.SimpleImputer(missing_values=np.nan, strategy="mean")
        ), 
        (
            "scaler", sklearn.compose.make_column_transformer(
                (
                    sklearn.preprocessing.MinMaxScaler(feature_range=(0,1)),
                    index_continous_columns
                ),
                remainder="passthrough"
            )
        ),
        (
            "classifier",
            implemented_classifiers[classifier]["classifier"],
        ),
    ]
)

#### Grid Search

In [ ]:
classifier_cv = sklearn.model_selection.GridSearchCV(
    estimator=pipeline,
    param_grid=implemented_classifiers[classifier]["param_grid"],
    scoring=score_metric,
)

### Model Training and Evaluation 

We use [mlflow](https://www.mlflow.org/) library to track each model training and save the resulting plots.


In [ ]:
mlflow.set_tracking_uri("file://" + str(ml_colon.OUTPUT_DIR / "mlruns"))
tracking_uri = mlflow.get_tracking_uri()
print("Current tracking uri: {}".format(tracking_uri)) # where the outputs are stored

mlflow.set_experiment("ml_colon")

In [ ]:
run_id = None
experiment_id = None
with mlflow.start_run() as run:

    run_id = run.info.run_id
    experiment_id = run.info.experiment_id
    print(f"Starting MlFlow run {run_id} for experiment {experiment_id}", "\n")
    mlflow.log_param("classifier", classifier)

    # The training of the model
    print(f"Training a {classifier} classifier")
    classifier_cv.fit(X_train, y_train)

    # Best parameters
    print("Best parameters:")
    print(classifier_cv.best_params_, "\n")

    mlflow.log_params(classifier_cv.best_params_)
    mlflow.log_metric(score_metric, classifier_cv.best_score_)

    # Evaluation of results
    print("Classification Report")
    y_pred = classifier_cv.predict(X_test)
    y_pred_proba = classifier_cv.predict_proba(X_test)
    report = sklearn.metrics.classification_report(y_test, y_pred, output_dict=True)

    IPython.display.display(pd.DataFrame(report).T)

    for label in ["0.0", "1.0"]:
        for k, v in report[label].items():
            mlflow.log_metric(f"label_{label}_{k}", v)


    print("Plotting Confusion Matrix")
    cm = sklearn.metrics.confusion_matrix(y_test, y_pred)
    cm_fig = sklearn.metrics.ConfusionMatrixDisplay(cm).plot()

    print("Plotting ROC curve")
    roc_fig = sklearn.metrics.plot_roc_curve(
        classifier_cv, X_test, y_test, name="ROC Curve"
    )

    print("Writing Artifacts to MlFlow")
    with tempfile.TemporaryDirectory() as tmp_dir:
        tmp_dir_path = pathlib.Path(tmp_dir)
        assert tmp_dir_path.exists()

        # Saving plots
        cm_fig.figure_.savefig(str(tmp_dir_path / "confusion_matrix.png"))
        roc_fig.figure_.savefig(str(tmp_dir_path / "roc.png"))

        # Saving Model
        with open(tmp_dir_path / "model.pkl", "wb") as model_file:
            pickle.dump(classifier_cv, model_file)           

        mlflow.log_artifacts(str(tmp_dir_path))

    # store trained model in list 
    trained_models.append(classifier_cv)



### Compare Different Models

Once we have a few models trained we can compare they Receiver Operator Curve (ROC) to see how well perform.



In [ ]:
fig, ax = plt.subplots(figsize=(12,9))

for model in trained_models:
    sklearn.metrics.plot_roc_curve(
    model, X_test, y_test, name=f"{model.best_estimator_.steps[-1][1]}", ax=ax
    )

 
#### MlFlow Dashboard

As explained in the `README.md` you can now run the MlFlow dashboard by executing
```
cd output/
mlflow ui
```
and then accessing the dashboard through your browser

### Loading a previously trained model

To load a previously trained model all one needs is to specify the run_id in which the model was trained.


In [ ]:
def load_model_from_run_id(experiment_id, run_id):
    model_path = ml_colon.OUTPUT_DIR / "mlruns" / str(experiment_id) / run_id / "artifacts" / "model.pkl" 
    if not model_path.exists():
        print(f"Could not find pickled model under {model_path}")
        raise FileNotFoundError
    with open(model_path, "rb") as model_file:
        return pickle.load(model_file)

In [ ]:

reloaded_model = load_model_from_run_id(experiment_id, run_id)

print(reloaded_model.estimator)